# GeoplotLib

- Mobility trace for the city of São Paulo
- URL: http://interscity.org/open_data/
- ZIP FILE: https://www.dropbox.com/s/hatzbujfspte12i/car-bus-simulation.zip?dl=0#

In [8]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np

# CSV DATASET from InterSCimulator
TRACE_DATA = '/home/tallys/git/master-research/src/../datasets/car-bus-simulation/output/events_lat_long.csv'
COLUMNS = ['time', 'action', 'vid', 'lat', 'lon']

# Define input size number of rows to be read from the csv into the pandas dataframe
UNITY=1
SMALL=10000
LARGE=1000000
BIG=None
INPUT_SIZE=LARGE

mdf = pd.read_csv(TRACE_DATA, nrows=INPUT_SIZE, names=COLUMNS, delimiter=";", header=None)
       
# All data points from a given vehicle id
def filter_by_id(id):
    return mdf.loc[mdf['vid'] == id]

# The first time a vehicle id registered an action in the simulation
def origin(id):
    return filter_by_id(id).loc[[0]]
        
# The last time a vehicle id registered an action in the simulation
def destiny(id):
    return filter_by_id(id).iloc[[-1]]

tool_tip = lambda x: 'time: ' + str(x["time"]) + ', lat: ' + str(x["lat"]) + ', lon: ' + str(x["lon"])

## a) Plot trajectory points of vehicle

- Select vehicle id (vid): 4858_52
- use geoplotlib.dot function to plot the data

In [2]:
import geoplotlib

points_df = filter_by_id('4858_52').reset_index()
start = origin('4858_52').reset_index()
end = destiny('4858_52').reset_index()

geoplotlib.dot(points_df, color="red", f_tooltip=tool_tip)
geoplotlib.dot(start, color="green", point_size=5, f_tooltip=tool_tip)
geoplotlib.dot(end, color="black", point_size=5, f_tooltip=tool_tip)
geoplotlib.show()

## b) Plot trajectory graph of vehicle

- Select vehicle id (vid): 4858_52
- build vehicle trajectory (dest_lat and dest_lon fields)
- use geoplotlib.graph function to plot the data

In [3]:
import geoplotlib

graph_df = filter_by_id('4858_52').reset_index()
start = origin('4858_52').reset_index()
end = destiny('4858_52').reset_index()

graph_df.loc[:, 'dest_lat'] = graph_df.shift(-1).loc[:, 'lat']
graph_df.loc[:, 'dest_lon'] = graph_df.shift(-1).loc[:, 'lon']
graph_df = graph_df[:-1]

# Edge colors are based on distance between points
geoplotlib.graph(graph_df, src_lat='lat', src_lon='lon', dest_lat='dest_lat', dest_lon='dest_lon', linewidth=80, alpha=255)

# Comment this line to plot the points
# geoplotlib.dot(graph_df, color="red", point_size=3, f_tooltip=tool_tip)
geoplotlib.dot(start, color="green", point_size=8, f_tooltip=tool_tip)
geoplotlib.dot(end, color="black", point_size=8, f_tooltip=tool_tip)
geoplotlib.show()



## c) Animated trajectory graph of vehicle

- Select vehicle id (vid): 4858_52
- build animated vehicle trajectory (dest_lat and dest_lon fields)
- use new layer class to plot the points on each new frame

In [4]:
# Custom layer
from geoplotlib.layers import BaseLayer
from geoplotlib.layers import HotspotManager
from geoplotlib.utils import BoundingBox
from geoplotlib.core import BatchPainter
import random
import time

class DotDensityLayer(BaseLayer):

    def __init__(self, data, color=None, point_size=2, f_tooltip=None):
        """Create a dot density map
        :param data: data access object
        :param color: color
        :param point_size: point size
        :param f_tooltip: function to return a tooltip string for a point
        """
        self.frame_counter = 0
        self.data = data
        self.color = color
        if self.color is None:
            self.color = [255,0,0]
        self.point_size = point_size
        self.f_tooltip = f_tooltip

        self.hotspots = HotspotManager()


    def invalidate(self, proj):
        self.x, self.y = proj.lonlat_to_screen(self.data['lon'], self.data['lat'])



    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter = BatchPainter()
        x, y = proj.lonlat_to_screen(self.data['lon'], self.data['lat'])
        if self.f_tooltip:
            for i in range(0, len(x)):
                record = {k: self.data[k][i] for k in self.data.keys()}
                self.hotspots.add_rect(x[i] - self.point_size, y[i] - self.point_size,
                                       2*self.point_size, 2*self.point_size,
                                       self.f_tooltip(record))
        #foo = ["blue", "red", "black"]

        #print(random.choice(foo))
        #self.color= random.choice(foo)
        self.painter.set_color(self.color)
        self.painter.points(x[self.frame_counter], y[self.frame_counter], 2*self.point_size, False)
        
        self.painter.batch_draw()
        picked = self.hotspots.pick(mouse_x, mouse_y)
        if picked:
            ui_manager.tooltip(picked)
        self.frame_counter += 1
        time.sleep(0.4)
        if self.frame_counter == len(x):
            self.frame_counter = 0


    def bbox(self):
        return BoundingBox.from_points(lons=self.data['lon'], lats=self.data['lat'])
        
geoplotlib.add_layer(DotDensityLayer(points_df))
geoplotlib.show()

## d) Plot dots animation of all lines

- use new layer class to plot the points on each new frame

In [5]:
# Custom layer
from geoplotlib.layers import BaseLayer
from geoplotlib.layers import HotspotManager
from geoplotlib.utils import BoundingBox
from geoplotlib.core import BatchPainter
import random
import time

trajectories = mdf

class DotDensityLayer(BaseLayer):

    def __init__(self, data, color=None, point_size=2, f_tooltip=None):
        """Create a dot density map
        :param data: data access object
        :param color: color
        :param point_size: point size
        :param f_tooltip: function to return a tooltip string for a point
        """
        self.frame_counter = 23
        self.data = data
        self.last = self.data.drop_duplicates(subset=['vid'], keep='first')
        self.color = color
        if self.color is None:
            self.color = [255,0,0]
        self.point_size = point_size
        self.f_tooltip = f_tooltip

        self.hotspots = HotspotManager()


    def invalidate(self, proj):
        self.last = self.data.loc[self.data['time'] == self.frame_counter]
        lon_at = self.last['lon']
        lat_at = self.last['lat']
        
        self.x, self.y = proj.lonlat_to_screen(lon_at, lat_at)
        


    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter = BatchPainter()
        current = self.data.loc[self.data['time'] == self.frame_counter]
        
        self.last = pd.concat([self.last, current]).drop_duplicates(subset='vid', keep='last')
        lon_at = self.last['lon']
        lat_at = self.last['lat']
        
        x, y = proj.lonlat_to_screen(lon_at, lat_at)
        
        if self.f_tooltip:
            for i in range(0, len(x)):
                record = {k: self.data[k][i] for k in self.data.keys()}
                self.hotspots.add_rect(x[i] - self.point_size, y[i] - self.point_size,
                                       2*self.point_size, 2*self.point_size,
                                       self.f_tooltip(record))
        foo = ["blue", "red", "black"]

        #self.color= random.choice(foo)
        self.painter.set_color(self.color)
        self.painter.points(x, y, 2*self.point_size, False)
        
        self.painter.batch_draw()
        picked = self.hotspots.pick(mouse_x, mouse_y)
        if picked:
            ui_manager.tooltip(picked)
        self.frame_counter += 1
        #time.sleep(0.4)


    def bbox(self):
        return BoundingBox.from_points(lons=self.data['lon'], lats=self.data['lat'])
        
geoplotlib.add_layer(DotDensityLayer(trajectories))
geoplotlib.show()

## e) Plot graph of all lines

- build vehicle trajectories of all lines (dest_lat and dest_lon fields)
- use geoplotlib.graph function to plot the data


In [ ]:
import geoplotlib

trajectories_graph = mdf

trajectories_graph.loc[:, 'dest_lat'] = trajectories_graph.shift(-1).loc[:, 'lat']
trajectories_graph.loc[:, 'dest_lon'] = trajectories_graph.shift(-1).loc[:, 'lon']
trajectories_graph = trajectories_graph[:-1]

# Edge colors are based on distance between points
geoplotlib.graph(trajectories_graph, src_lat='lat', src_lon='lon', dest_lat='dest_lat', dest_lon='dest_lon', linewidth=80, alpha=255)
geoplotlib.show()
#trajectories[self.frame_counter]

In [7]:


#help(trajectories['vid'].unique)
#trajectories['action'].unique()
last = trajectories.drop_duplicates(subset=['vid'])
last.reset_index()
#last.loc[last['time'] == 25]
xablau = pd.concat([last, trajectories.loc[trajectories['time'] == 31]])
new_xablau = xablau.drop_duplicates(subset='vid', keep='last')
new_xablau
#help(xablau.drop_duplicates)

#help(trajectories.loc)
#trajectories
#pd.concat([trajectories, trajectories])


,time,action,vid,lat,lon,dest_lat,dest_lon
1,27,start,4858_73,-23.624235,-46.648388,-23.535976,-46.635610
2,27,start,94_17,-23.535976,-46.635610,-23.624077,-46.648453
5,35,start,8394_43,-23.561275,-46.695423,-23.561200,-46.695156
7,41,start,9474_14,-23.542536,-46.685204,-23.543137,-46.684284
8,42,move,9474_41,-23.543137,-46.684284,-23.561563,-46.694824
12,51,start,8000_16,-23.562016,-46.680690,-23.624235,-46.648388
13,52,start,4858_54,-23.624235,-46.648388,-23.624329,-46.649160
16,54,start,4858_77,-23.624235,-46.648388,-23.624329,-46.649160
19,56,start,4858_33,-23.624235,-46.648388,-23.562902,-46.694500
22,59,start,9474_43,-23.542536,-46.685204,-23.624077,-46.648453
